In [ ]:
# Revenue Leakage Analysis

This notebook analyzes the transaction data to identify and quantify revenue leakage patterns. It supports the dashboard visualization to help identify areas of concern for large enterprises.

## Objectives
1. Identify patterns in revenue leakage
2. Quantify financial impact of different leakage types
3. Build predictive models to detect high-risk transactions
4. Generate insights for the dashboard visualization


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import os
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Set plotting style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Configure visualization settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# Function to format currency
def format_currency(amount):
    """Format numbers as currency"""
    return f"₹{amount:,.2f}"


In [ ]:
# Load the transaction data
print("Loading transaction data...")

try:
    # Try to load the processed data first
    df = pd.read_csv("../data/processed/cleaned_transactions.csv")
    print("Loaded processed data successfully!")
except FileNotFoundError:
    # If processed data doesn't exist, load the raw data
    print("Processed data not found. Loading raw data instead...")
    df = pd.read_csv("../data/raw/transactions.csv")
    print("Loaded raw data successfully!")

# Display basic information about the dataset
print(f"\nDataset shape: {df.shape}")
print("\nColumn information:")
df.info()


In [ ]:
# Data preparation
print("Preparing data for analysis...")

# Convert date columns to datetime
date_columns = ['Invoice_Date', 'Due_Date', 'Payment_Date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Calculate additional features if they don't exist
if 'Payment_Delay_Days' not in df.columns:
    df['Payment_Delay_Days'] = (df['Payment_Date'] - df['Due_Date']).dt.days
    
if 'Expected_Payment' not in df.columns:
    df['Expected_Payment'] = df['Amount_Billed'] - df['Discount']
    
if 'Payment_Gap' not in df.columns:
    df['Payment_Gap'] = df['Expected_Payment'] - df['Amount_Received']

# Fill missing values in Amount_Received for missing payments
df.loc[df['Payment_Date'].isna(), 'Amount_Received'] = 0.0

# Create a monthly period column
df['Invoice_Month'] = df['Invoice_Date'].dt.to_period('M').astype(str)

# Examine the first few rows
df.head()


In [ ]:
# 1. Revenue Leakage Summary

# Calculate total billed, received and leaked amounts
total_billed = df['Amount_Billed'].sum()
total_received = df['Amount_Received'].sum()
total_leaked = total_billed - total_received
leakage_percent = (total_leaked / total_billed) * 100

print(f"Total Amount Billed:  {format_currency(total_billed)}")
print(f"Total Amount Received: {format_currency(total_received)}")
print(f"Total Revenue Leakage: {format_currency(total_leaked)}")
print(f"Leakage Percentage:    {leakage_percent:.2f}%")

# Analyze leakage by types
if 'Leakage_Type' in df.columns and 'Is_Leaked' in df.columns:
    leaked_df = df[df['Is_Leaked'] == 1]
    leakage_by_type = leaked_df.groupby('Leakage_Type').agg({
        'Invoice_ID': 'count',
        'Amount_Billed': 'sum',
        'Amount_Received': 'sum'
    }).reset_index()
    
    leakage_by_type['Leakage_Amount'] = leakage_by_type['Amount_Billed'] - leakage_by_type['Amount_Received']
    leakage_by_type['Percentage'] = (leakage_by_type['Leakage_Amount'] / leakage_by_type['Leakage_Amount'].sum()) * 100
    leakage_by_type = leakage_by_type.sort_values('Leakage_Amount', ascending=False)
    
    print("\nLeakage by Type:")
    print(leakage_by_type[['Leakage_Type', 'Invoice_ID', 'Leakage_Amount', 'Percentage']]
          .rename(columns={'Invoice_ID': 'Count'}))
    
    # Create a pie chart for leakage distribution
    plt.figure(figsize=(10, 6))
    plt.pie(leakage_by_type['Leakage_Amount'], 
            labels=leakage_by_type['Leakage_Type'], 
            autopct='%1.1f%%',
            startangle=90,
            shadow=True)
    plt.title('Distribution of Revenue Leakage by Type')
    plt.axis('equal')
    plt.show()


In [ ]:
# 2. Payment Delay Analysis

# Create delay categories
df['Delay_Status'] = 'On Time'
df.loc[df['Payment_Date'].isna(), 'Delay_Status'] = 'Missing'
df.loc[(df['Payment_Delay_Days'] > 0) & (df['Payment_Date'].notna()), 'Delay_Status'] = 'Delayed'

# Get statistics on payment delays
delay_stats = df.groupby('Delay_Status').agg({
    'Invoice_ID': 'count',
    'Amount_Billed': 'sum'
}).reset_index()

delay_stats['Percentage'] = delay_stats['Invoice_ID'] / delay_stats['Invoice_ID'].sum() * 100
print("Payment Status Distribution:")
print(delay_stats)

# For delayed payments, analyze the delay distribution
if len(df[df['Delay_Status'] == 'Delayed']) > 0:
    delayed_df = df[df['Delay_Status'] == 'Delayed']
    
    plt.figure(figsize=(12, 6))
    
    # Create a histogram of delay days
    plt.subplot(1, 2, 1)
    sns.histplot(delayed_df['Payment_Delay_Days'], bins=20, kde=True)
    plt.title('Distribution of Payment Delay Days')
    plt.xlabel('Delay (Days)')
    plt.ylabel('Count')
    
    # Create delay categories for easier analysis
    delay_categories = [
        (0, 15, '1-15 days'),
        (15, 30, '16-30 days'),
        (30, 60, '31-60 days'),
        (60, float('inf'), '60+ days')
    ]
    
    for min_days, max_days, label in delay_categories:
        mask = (delayed_df['Payment_Delay_Days'] > min_days) & (delayed_df['Payment_Delay_Days'] <= max_days)
        delayed_df.loc[mask, 'Delay_Category'] = label
    
    # Plot delay categories
    plt.subplot(1, 2, 2)
    delay_counts = delayed_df['Delay_Category'].value_counts().sort_index()
    sns.barplot(x=delay_counts.index, y=delay_counts.values)
    plt.title('Payment Delay Categories')
    plt.xlabel('Delay Category')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Is there a correlation between delay days and invoice amount?
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=delayed_df, x='Payment_Delay_Days', y='Amount_Billed')
    plt.title('Relationship Between Invoice Amount and Payment Delay')
    plt.xlabel('Delay (Days)')
    plt.ylabel('Invoice Amount (₹)')
    plt.show()


In [ ]:
# 3. Customer Analysis

# Top customers by leakage amount
customer_leakage = df.groupby('Customer_ID').agg({
    'Invoice_ID': 'count',
    'Amount_Billed': 'sum',
    'Amount_Received': 'sum'
}).reset_index()

customer_leakage['Leakage'] = customer_leakage['Amount_Billed'] - customer_leakage['Amount_Received']
customer_leakage['Leakage_Percent'] = (customer_leakage['Leakage'] / customer_leakage['Amount_Billed']) * 100
customer_leakage_sorted = customer_leakage.sort_values('Leakage', ascending=False).head(10)

print("Top 10 Customers by Revenue Leakage:")
print(customer_leakage_sorted[['Customer_ID', 'Leakage', 'Leakage_Percent', 'Invoice_ID']]
      .rename(columns={'Invoice_ID': 'Invoice_Count'}))

# Visualize top customers by leakage amount
plt.figure(figsize=(12, 6))
sns.barplot(x='Customer_ID', y='Leakage', data=customer_leakage_sorted)
plt.title('Top 10 Customers by Revenue Leakage')
plt.xlabel('Customer ID')
plt.ylabel('Leakage Amount (₹)')
plt.xticks(rotation=45)
for i, v in enumerate(customer_leakage_sorted['Leakage']):
    plt.text(i, v + 500, f'{v:,.0f}', ha='center')
plt.tight_layout()
plt.show()

# Customer risk assessment - identify risky customers
if 'Is_Leaked' in df.columns:
    customer_risk = df.groupby('Customer_ID').agg({
        'Invoice_ID': 'count',
        'Is_Leaked': 'sum',
        'Amount_Billed': 'sum',
        'Amount_Received': 'sum'
    }).reset_index()
    
    customer_risk['Leakage'] = customer_risk['Amount_Billed'] - customer_risk['Amount_Received']
    customer_risk['Leakage_Percent'] = (customer_risk['Leakage'] / customer_risk['Amount_Billed']) * 100
    customer_risk['Leak_Rate'] = (customer_risk['Is_Leaked'] / customer_risk['Invoice_ID']) * 100
    
    # Assign risk score based on leakage percentage and leak rate
    customer_risk['Risk_Score'] = (customer_risk['Leakage_Percent'] * 0.7) + (customer_risk['Leak_Rate'] * 0.3)
    high_risk_customers = customer_risk.sort_values('Risk_Score', ascending=False).head(10)
    
    print("\nTop 10 High-Risk Customers:")
    print(high_risk_customers[['Customer_ID', 'Risk_Score', 'Leakage_Percent', 'Leak_Rate']])
    
    # Scatter plot of customer risk - bubble size represents amount billed
    plt.figure(figsize=(12, 8))
    sns.scatterplot(data=customer_risk, 
                   x='Leak_Rate', 
                   y='Leakage_Percent', 
                   size='Amount_Billed',
                   hue='Risk_Score',
                   palette='YlOrRd',
                   sizes=(20, 500),
                   legend='brief')
    
    # Add labels for high-risk customers
    for _, row in high_risk_customers.iterrows():
        plt.annotate(row['Customer_ID'], 
                    xy=(row['Leak_Rate'], row['Leakage_Percent']),
                    xytext=(5, 5),
                    textcoords='offset points',
                    fontsize=9)
    
    plt.title('Customer Risk Assessment')
    plt.xlabel('Leak Rate (% of Invoices with Leakage)')
    plt.ylabel('Leakage Percentage (% of Billed Amount)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
# 4. Time Trend Analysis

# Monthly trend of revenue and leakage
monthly_data = df.groupby(df['Invoice_Date'].dt.to_period('M').astype(str)).agg({
    'Invoice_ID': 'count',
    'Amount_Billed': 'sum',
    'Amount_Received': 'sum'
}).reset_index()

monthly_data['Leakage'] = monthly_data['Amount_Billed'] - monthly_data['Amount_Received']
monthly_data['Leakage_Percent'] = (monthly_data['Leakage'] / monthly_data['Amount_Billed']) * 100
monthly_data = monthly_data.sort_values('Invoice_Date')

print("Monthly Revenue and Leakage Trends:")
print(monthly_data[['Invoice_Date', 'Amount_Billed', 'Amount_Received', 'Leakage', 'Leakage_Percent']])

# Visualize the monthly trend
plt.figure(figsize=(15, 8))

# Plot 1: Revenue trend
plt.subplot(2, 1, 1)
plt.plot(monthly_data['Invoice_Date'], monthly_data['Amount_Billed'], marker='o', label='Amount Billed')
plt.plot(monthly_data['Invoice_Date'], monthly_data['Amount_Received'], marker='s', label='Amount Received')
plt.fill_between(monthly_data['Invoice_Date'], 
                 monthly_data['Amount_Billed'], 
                 monthly_data['Amount_Received'], 
                 alpha=0.3, 
                 color='red',
                 label='Leakage')
plt.title('Monthly Revenue Trend')
plt.xlabel('Month')
plt.ylabel('Amount (₹)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Plot 2: Leakage percentage trend
plt.subplot(2, 1, 2)
plt.bar(monthly_data['Invoice_Date'], monthly_data['Leakage_Percent'], color='orangered')
plt.title('Monthly Leakage Percentage')
plt.xlabel('Month')
plt.ylabel('Leakage Percentage (%)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Create a line chart using Plotly for interactive visualization
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_data['Invoice_Date'],
    y=monthly_data['Amount_Billed'],
    mode='lines+markers',
    name='Amount Billed',
    line=dict(color='royalblue', width=2)
))

fig.add_trace(go.Scatter(
    x=monthly_data['Invoice_Date'],
    y=monthly_data['Amount_Received'],
    mode='lines+markers',
    name='Amount Received',
    line=dict(color='green', width=2)
))

fig.add_trace(go.Bar(
    x=monthly_data['Invoice_Date'],
    y=monthly_data['Leakage'],
    name='Leakage',
    marker_color='crimson'
))

fig.update_layout(
    title='Monthly Revenue and Leakage Trend',
    xaxis_title='Month',
    yaxis_title='Amount (₹)',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template='plotly_white'
)

fig.show()


In [ ]:
# 5. Anomaly Detection using Machine Learning

print("Performing anomaly detection using machine learning...")

# Prepare features for anomaly detection
features = ['Amount_Billed', 'Discount']
if 'Payment_Delay_Days' in df.columns:
    # Fill missing values with a large number to indicate missing payments
    df['Payment_Delay_Days_Filled'] = df['Payment_Delay_Days'].fillna(365)
    features.append('Payment_Delay_Days_Filled')
    
if 'Payment_Gap' in df.columns:
    df['Payment_Gap_Filled'] = df['Payment_Gap'].fillna(df['Amount_Billed'])
    features.append('Payment_Gap_Filled')

if 'Discount' in df.columns:
    df['Discount_Percentage'] = (df['Discount'] / df['Amount_Billed'] * 100).fillna(0)
    features.append('Discount_Percentage')

# Create a feature matrix
X = df[features].copy()

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply Isolation Forest for anomaly detection
print("Applying Isolation Forest for anomaly detection...")
iso_forest = IsolationForest(contamination=0.1, random_state=42)
df['anomaly_scores'] = iso_forest.fit_predict(X_scaled)
df['is_anomaly'] = df['anomaly_scores'] == -1

# Get anomaly detection results
anomalies = df[df['is_anomaly']]
print(f"Detected {len(anomalies)} anomalies out of {len(df)} transactions ({len(anomalies)/len(df)*100:.2f}%)")

# Compare with known leakage cases
if 'Is_Leaked' in df.columns:
    true_leakage = df[df['Is_Leaked'] == 1]
    detected_leakage = anomalies[anomalies['Is_Leaked'] == 1]
    
    # Calculate detection metrics
    precision = len(detected_leakage) / len(anomalies) if len(anomalies) > 0 else 0
    recall = len(detected_leakage) / len(true_leakage) if len(true_leakage) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\nAnomaly Detection Performance:")
    print(f"Precision: {precision:.2f} (what % of detected anomalies are actual leakage cases)")
    print(f"Recall: {recall:.2f} (what % of actual leakage cases were detected)")
    print(f"F1 Score: {f1_score:.2f}")
    
    # Confusion matrix
    tn = len(df[(df['is_anomaly'] == False) & (df['Is_Leaked'] == 0)])
    fp = len(df[(df['is_anomaly'] == True) & (df['Is_Leaked'] == 0)])
    fn = len(df[(df['is_anomaly'] == False) & (df['Is_Leaked'] == 1)])
    tp = len(df[(df['is_anomaly'] == True) & (df['Is_Leaked'] == 1)])
    
    print("\nConfusion Matrix:")
    print(f"True Positives: {tp} (correctly detected leakage)")
    print(f"False Positives: {fp} (incorrectly flagged as anomaly)")
    print(f"True Negatives: {tn} (correctly identified as normal)")
    print(f"False Negatives: {fn} (missed leakage cases)")

# Visualize the anomalies
plt.figure(figsize=(12, 8))

# Choose two features for visualization
x_feature = 'Amount_Billed'
y_feature = 'Payment_Delay_Days_Filled' if 'Payment_Delay_Days_Filled' in df.columns else 'Discount_Percentage'

plt.scatter(df[~df['is_anomaly']][x_feature], 
            df[~df['is_anomaly']][y_feature], 
            label='Normal',
            alpha=0.5,
            s=50)

plt.scatter(df[df['is_anomaly']][x_feature], 
            df[df['is_anomaly']][y_feature], 
            label='Anomaly',
            color='red',
            marker='x',
            s=100)

if 'Is_Leaked' in df.columns:
    plt.scatter(df[(df['Is_Leaked'] == 1) & (~df['is_anomaly'])][x_feature], 
                df[(df['Is_Leaked'] == 1) & (~df['is_anomaly'])][y_feature], 
                label='Missed Leakage',
                color='orange',
                marker='o',
                s=80,
                edgecolors='black')

plt.title('Anomaly Detection Results')
plt.xlabel(x_feature)
plt.ylabel(y_feature)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Display top anomalies
print("\nTop 10 Anomalies with Highest Risk:")
risk_cols = ['Invoice_ID', 'Customer_ID', 'Amount_Billed', 'Amount_Received', 
             'Payment_Delay_Days', 'Discount_Percentage', 'Is_Leaked', 'Leakage_Type']
risk_cols = [col for col in risk_cols if col in df.columns]
print(anomalies.sort_values('Amount_Billed', ascending=False).head(10)[risk_cols])


In [ ]:
# 6. Conclusions and Recommendations

print("Summary of Revenue Leakage Analysis:")
print(f"Total Billed Amount: {format_currency(total_billed)}")
print(f"Total Received Amount: {format_currency(total_received)}")
print(f"Total Leakage Amount: {format_currency(total_leaked)}")
print(f"Leakage Percentage: {leakage_percent:.2f}%")

if 'Leakage_Type' in df.columns:
    # Most common leakage types
    leakage_counts = df[df['Is_Leaked'] == 1]['Leakage_Type'].value_counts()
    print("\nMost Common Leakage Types:")
    for leakage_type, count in leakage_counts.items():
        print(f"  {leakage_type}: {count} invoices")
    
    # Most costly leakage types
    leakage_costs = df[df['Is_Leaked'] == 1].groupby('Leakage_Type').agg({
        'Amount_Billed': 'sum',
        'Amount_Received': 'sum'
    })
    leakage_costs['Leakage_Amount'] = leakage_costs['Amount_Billed'] - leakage_costs['Amount_Received']
    leakage_costs = leakage_costs.sort_values('Leakage_Amount', ascending=False)
    
    print("\nMost Costly Leakage Types:")
    for leakage_type, row in leakage_costs.iterrows():
        print(f"  {leakage_type}: {format_currency(row['Leakage_Amount'])}")

# Key insights
print("\nKey Insights:")
print("1. Overall Revenue Leakage Impact:")
print(f"   - The total revenue leakage of {format_currency(total_leaked)} represents {leakage_percent:.2f}% of the total billed amount.")
print(f"   - This suggests significant opportunity for revenue recovery.")

print("\n2. Customer Analysis:")
print("   - High-risk customers have been identified who consistently experience revenue leakage.")
print("   - Top customers by leakage amount may require specific attention and customized collection strategies.")

print("\n3. Payment Delay Patterns:")
print("   - Payment delays contribute significantly to cash flow issues even when payments are eventually received.")
print("   - Time trend analysis shows patterns in payment behavior that can be predicted and managed.")

print("\n4. Machine Learning Applications:")
print("   - Anomaly detection can identify unusual transactions that may represent leakage.")
print("   - Predictive models could be developed to forecast high-risk invoices before they become problematic.")

print("\nRecommendations for Reducing Revenue Leakage:")
print("1. Implement automated invoice reconciliation to quickly identify discrepancies.")
print("2. Create a structured follow-up process for overdue payments with escalation paths.")
print("3. Review discount policies and approval processes to prevent over-discounting.")
print("4. Deploy the dashboard for real-time monitoring of leakage metrics.")
print("5. Establish customer risk tiers with differentiated payment terms based on risk profiles.")
print("6. Integrate the anomaly detection algorithm into the invoice workflow to flag potential issues early.")
print("7. Conduct regular audits of the highest-risk customers and transaction types.")
